In [1]:
!pip install langchain langchain-experimental openai pymysql

# Importing Necessary Packages

In [2]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

In [3]:
OPENAI_API_KEY = "Your API Key"
llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

C:\Users\akhil\AppData\Local\Temp\ipykernel_19556\1703113971.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


# Database setup

In [13]:
host = 'localhost'
port = '3306'
username = 'root'
# password 
database_schema = 'mydb'
mysql_uri = f"mysql+pymysql://{username}@{host}:{port}/{database_schema}"
# mysql_uri = f"mysql+pymysql://{username:password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mysql_uri, include_tables=['p6_uk_bank_customers'],sample_rows_in_table_info=2)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [14]:
def retrieve_from_db(query: str) -> str:
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

In [17]:
def generate(query: str) -> str:
    db_context = retrieve_from_db(query)
    
    system_message = """You are a database expert specializing in customer segmentation, balance prediction, or understanding customer demographics.
        your task is to answer users' questions by providing relevant information from a database of training data.
        Tha database contaims the following information for each individual:
        -Customer ID: A unique identifier for each customer(discrete)
        -Name: The first name of the customer(categorical)
        -Surname: The last name of the customer(categorical)
        -Gender: The gender of the customer (discrete) (Male/Female)
        -Age: The age of the customer(discrete)
        -Region: The region where the customer resides(categorical)
        -Job Classification: The type of job ( White Collar, Blue Collar)(categorical)
        -Date Joined: The date the customer joined the bank(categorical)
        -Balance: The current balance in the customer's account(continous)

        Example:

        Input:
        Which region has the most customers?

        context:
        Analyzing customer distribution across regions helps the bank identify its strongest customer base geographically.
        
        Output:
        The region with the most customers is England, with 2,159 customers
        """
    human_qry_template = HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        
        Context:
        {db_context}
        
        Output:
        """
    )
    messages = [
      SystemMessage(content=system_message),
      human_qry_template.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

# Ask queries

In [18]:
generate("Provide names of all people from England?")



> Entering new SQLDatabaseChain chain...
Provide names of all people from England?
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Region` = 'England';
SQLResult: [('Simon',), ('Liam',), ('Deirdre',), ('Dorothy',), ('Dominic',), ('Dominic',), ('Benjamin',), ('Ryan',), ('Nicholas',), ('William',), ('Audrey',), ('Jennifer',), ('Alison',), ('Luke',), ('Megan',), ('Jake',), ('Karen',), ('Yvonne',), ('Sophie',), ('Robert',), ('Edward',), ('Diane',), ('Dorothy',), ('Simon',), ('Charles',), ('Sophie',), ('Kimberly',), ('Madeleine',), ('Irene',), ('Cameron',), ('Liam',), ('Donna',), ('Piers',), ('Phil',), ('Ryan',), ('Kylie',), ('Fiona',), ('Kevin',), ('Bernadette',), ('Frank',), ('Diane',), ('David',), ('Joan',), ('Neil',), ('Paul',), ('Dylan',), ('Pippa',), ('Matt',), ('Jack',), ('Jasmine',), ('Michelle',), ('Wanda',), ('Blake',), ('Jane',), ('Dan',), ('Lisa',), ('Rebecca',), ('Joshua',), ('Heather',), ('Maria',), ('Alison',), ('Amy',), ('Vanessa',), ('Andrea',), ('Ian',), ('Jacob'

'The names of all people from England are Simon, Liam, Deirdre, Dorothy, and Dominic.'

In [19]:
generate("Provide count of all Male people")




> Entering new SQLDatabaseChain chain...
Provide count of all Male people
SQLQuery:SELECT COUNT(`Customer ID`) AS Male_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Male';
SQLResult: [(2165,)]
Answer:There are 2165 Male people in the database.
> Finished chain.


'The count of all Male people in the database is 2,165.'

In [20]:
generate("Provide count of all female who are from Wales")




> Entering new SQLDatabaseChain chain...
Provide count of all female who are from Wales
SQLQuery:SELECT COUNT(`Customer ID`) AS FemaleCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND Region = 'Wales'
SQLResult: [(282,)]
Answer:There are 282 female customers from Wales.
> Finished chain.


'There are 282 female customers from Wales.'

In [21]:
generate("Provide count of people with White collar jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with White collar jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `White Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar';
SQLResult: [(1955,)]
Answer:There are 1955 people with White collar jobs.
> Finished chain.


'The count of people with White collar jobs is 1,955.'

In [23]:
generate("Provide count of people with Blue Color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people with Blue Color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Blue Collar Count`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'Blue Collar';
SQLResult: [(1049,)]
Answer:Blue Collar Count: 1
> Finished chain.


'The count of people with Blue Collar jobs is 1.'

In [24]:
generate("Provide count of people whose balance is above 5000")



> Entering new SQLDatabaseChain chain...
Provide count of people whose balance is above 5000
SQLQuery:SELECT COUNT(`Customer ID`) AS `Count`
FROM p6_uk_bank_customers
WHERE `Balance` > 5000;
SQLResult: [(3690,)]
Answer:There are 3690 people whose balance is above 5000.
> Finished chain.


'The count of people whose balance is above 5000 is 3690.'

In [25]:
generate("Provide names of all people who are white color and age above 35")



> Entering new SQLDatabaseChain chain...
Provide names of all people who are white color and age above 35
SQLQuery:SELECT `Name`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar' AND `Age` > 35
LIMIT 5;
SQLResult: [('Liam',), ('Dominic',), ('Dominic',), ('Benjamin',), ('Nicholas',)]
Answer:Liam, Dominic, Benjamin, Nicholas
> Finished chain.


'The names of people who are white collar and above the age of 35 are Dominic and Nicholas.'

In [26]:
generate("Provide count of all female people who are with White collar job")



> Entering new SQLDatabaseChain chain...
Provide count of all female people who are with White collar job
SQLQuery:SELECT COUNT(`Customer ID`) AS Female_White_Collar_Count
FROM p6_uk_bank_customers
WHERE Gender = 'Female' AND `Job Classification` = 'White Collar';
SQLResult: [(1010,)]
Answer:There are 1010 female customers with a White Collar job.
> Finished chain.


'There are 1010 female customers with a White Collar job.'

In [27]:
generate("Provide count of people from Wales and doing white color jobs")



> Entering new SQLDatabaseChain chain...
Provide count of people from Wales and doing white color jobs
SQLQuery:SELECT COUNT(`Customer ID`) AS `Wales_White_Collar_Count`
FROM p6_uk_bank_customers
WHERE Region = 'Wales' AND `Job Classification` = 'White Collar';
SQLResult: [(305,)]
Answer:There are 305 people from Wales who are doing white collar jobs.
> Finished chain.


'The count of people from Wales doing white collar jobs is 305.'

In [28]:
generate("Provide average age of people with white color job")



> Entering new SQLDatabaseChain chain...
Provide average age of people with white color job
SQLQuery:SELECT AVG(`Age`) AS `Average Age`
FROM p6_uk_bank_customers
WHERE `Job Classification` = 'White Collar'
SQLResult: [(Decimal('35.2793'),)]
Answer:The average age of people with a white collar job is 35.2793.
> Finished chain.


'The average age of people with a white collar job is 35.2793.'

In [44]:
generate("provide maximum balance and name of the person")



> Entering new SQLDatabaseChain chain...
provide maximum balance and name of the person
SQLQuery:SELECT `Name`, `Balance` 
FROM p6_uk_bank_customers 
ORDER BY `Balance` DESC 
LIMIT 1;
SQLResult: [('Dorothy', Decimal('183467.70'))]
Answer:The person with the maximum balance is Dorothy with a balance of £183,467.70.
> Finished chain.


'The person with the maximum balance is Dorothy with a balance of £183,467.70.'

In [45]:
generate("Provide minimum balance and name, job classification")



> Entering new SQLDatabaseChain chain...
Provide minimum balance and name, job classification
SQLQuery:SELECT `Name`, `Job Classification`, MIN(`Balance`) AS `Minimum Balance`
FROM p6_uk_bank_customers
GROUP BY `Name`, `Job Classification`
ORDER BY `Minimum Balance`
LIMIT 5;
SQLResult: [('Jane', 'White Collar', Decimal('11.52')), ('Olivia', 'White Collar', Decimal('21.03')), ('Warren', 'Blue Collar', Decimal('69.01')), ('Megan', 'Blue Collar', Decimal('69.78')), ('Stewart', 'Blue Collar', Decimal('77.46'))]
Answer:The customers with the minimum balance along with their names and job classifications are:
1. Jane - White Collar (Minimum Balance: 11.52)
2. Olivia - White Collar (Minimum Balance: 21.03)
3. Warren - Blue Collar (Minimum Balance: 69.01)
4. Megan - Blue Collar (Minimum Balance: 69.78)
5. Stewart - Blue Collar (Minimum Balance: 77.46)
> Finished chain.


'The customers with the minimum balance along with their names and job classifications are:\n1. Jane - White Collar (Minimum Balance: 11.52)\n2. Olivia - White Collar (Minimum Balance: 21.03)\n3. Warren - Blue Collar (Minimum Balance: 69.01)\n4. Megan - Blue Collar (Minimum Balance: 69.78)\n5. Stewart - Blue Collar (Minimum Balance: 77.46)'

In [46]:
generate("who is the join in the date of 12 jan 15")



> Entering new SQLDatabaseChain chain...
who is the join in the date of 12 jan 15
SQLQuery:SELECT `Name`, `Surname`
FROM p6_uk_bank_customers
WHERE `Date Joined` = '12.Jan.15'
LIMIT 5;
SQLResult: [('Dominic', 'Parr'), ('Dominic', 'Lewis'), ('Benjamin', 'Grant'), ('Ryan', 'MacDonald'), ('Thomas', 'Lawrence')]
Answer:Dominic Parr, Dominic Lewis, Benjamin Grant, Ryan MacDonald, Thomas Lawrence
> Finished chain.


"I'm sorry, but I couldn't find any information related to customers who joined on the date of 12th January 2015 in the provided context."

In [33]:
generate("provide me names of lowest age people")



> Entering new SQLDatabaseChain chain...
provide me names of lowest age people
SQLQuery:SELECT `Name`, `Age`
FROM p6_uk_bank_customers
ORDER BY `Age` ASC
LIMIT 5;
SQLResult: [('Jennifer', 15), ('Gabrielle', 17), ('Dominic', 17), ('Bernadette', 17), ('Olivia', 17)]
Answer:Jennifer, Gabrielle, Dominic, Bernadette, and Olivia are the names of the people with the lowest age in the database.
> Finished chain.


'The names of the people with the lowest age in the database are Jennifer, Gabrielle, Dominic, Bernadette, and Olivia.'

In [47]:
generate("which person balance is lowest one")



> Entering new SQLDatabaseChain chain...
which person balance is lowest one
SQLQuery:SELECT `Name`, `Surname`, `Balance`
FROM p6_uk_bank_customers
ORDER BY `Balance` ASC
LIMIT 1;
SQLResult: [('Jane', 'King', Decimal('11.52'))]
Answer:The person with the lowest balance is Jane King with a balance of 11.52.
> Finished chain.


'The person with the lowest balance is Jane King with a balance of 11.52.'

In [37]:
generate("how many female poeple are there")



> Entering new SQLDatabaseChain chain...
how many female poeple are there
SQLQuery:SELECT COUNT(*) AS FemaleCount
FROM p6_uk_bank_customers
WHERE Gender = 'Female';
SQLResult: [(1849,)]
Answer:There are 1849 female people.
> Finished chain.


'There are 1,849 female people.'

In [38]:
generate("provide me the surnames of male people")



> Entering new SQLDatabaseChain chain...
provide me the surnames of male people
SQLQuery:SELECT `Surname`
FROM p6_uk_bank_customers
WHERE `Gender` = 'Male'
LIMIT 5;
SQLResult: [('Walsh',), ('Brown',), ('Parr',), ('Parr',), ('Lewis',)]
Answer:Walsh, Brown, Parr, Parr, Lewis
> Finished chain.


'The surnames of male people are Walsh, Brown, Parr, Parr, and Lewis.'

In [40]:
generate("which person surname  length is small")



> Entering new SQLDatabaseChain chain...
which person surname  length is small
SQLQuery:SELECT `Surname` 
FROM p6_uk_bank_customers 
ORDER BY LENGTH(`Surname`) ASC 
LIMIT 5;
SQLResult: [('May',), ('Lee',), ('May',), ('Lee',), ('May',)]
Answer:The surnames with the smallest length are May and Lee.
> Finished chain.


'The surnames with the smallest length are May and Lee.'

In [42]:
generate("whose customer id is easy one")



> Entering new SQLDatabaseChain chain...
whose customer id is easy one
SQLQuery:SELECT `Customer ID`, `Name`, `Surname` 
FROM p6_uk_bank_customers 
WHERE `Customer ID` = 100000001
LIMIT 1;
SQLResult: [(100000001, 'Simon', 'Walsh')]
Answer:Customer ID 100000001 belongs to Simon Walsh.
> Finished chain.


'Customer ID 100000001 belongs to Simon Walsh.'

In [43]:
generate("provide me customer id's who are from scotland")



> Entering new SQLDatabaseChain chain...
provide me customer id's who are from scotland
SQLQuery:SELECT `Customer ID`
FROM p6_uk_bank_customers
WHERE Region = 'Scotland'
LIMIT 5;
SQLResult: [(200000008,), (200000014,), (200000017,), (200000018,), (200000022,)]
Answer:Customer ID's of customers who are from Scotland are 200000008, 200000014, 200000017, 200000018, and 200000022.
> Finished chain.


"Customer ID's of customers who are from Scotland are 200000008, 200000014, 200000017, 200000018, and 200000022."